In [ ]:
from nbfinder import NotebookFinder
import sys
sys.meta_path.append(NotebookFinder())
import pickle
from notebooks.metrics.objectives import *
from os.path import join, exists
from os import makedirs, mkdir
from notebooks.plotting.curve_plotter import plot_roc_curve, plot_learn_curve
from notebooks.util import makedir_if_not_there, iterate_minibatches, save_weights
from notebooks.metrics.metrics_processor_caffe import MetricsProcessor
from notebooks.printing.print_utils import *
import time
from lasagne.layers import *
import numpy as np

class TrainVal(object):
    def __init__(self, kwargs, fns, networks):
        self.kwargs = kwargs
        self.fns = fns
        self.epoch = 0
        self.networks = networks
        print_network(networks, self.kwargs)
        self.mp = MetricsProcessor(kwargs)
        self.epoch_time = 0

        
    def train(self):
        for epoch in range(self.kwargs["num_epochs"]):
            self.train_one_epoch()

            
    def test(self):
        self._do_one_epoch(type_="test")
        print_results(self.kwargs, self.epoch, self.mp.metrics)
    
    
    def train_one_epoch(self):
        self._do_one_epoch(type_="train")
        self._do_one_epoch(type_="validation")
        save_weights(self.mp.metrics, self.kwargs, self.networks)
        plot_learn_curve(self.mp.metrics, self.kwargs["save_path"])
        print_results(self.kwargs, self.epoch, self.mp.metrics)
        self.epoch += 1
        
        
    def _do_one_epoch(self, type_="train"):
        print("beginning epoch %i %s" % (self.epoch, type_))
        self.do_learn_loop(type_)
        self.postprocess(type_)
        
        
    def do_learn_loop(self,type_):
        start_time = time.time()
        batches = 0
        for minibatch in self.kwargs[type_ +"_iterator"]:
            batch_time = time.time()
            x,y,w = [minibatch[k] for k in ["data", "label", "normweight"]]
            loss = self.fns[type_](x,y,w)
            acc = self.fns["acc"](x,y,w)
            self.mp.add_metrics(dict(loss=loss, acc=acc))
            batches += 1
            print("batch number: ",batches," time[s] = ",time.time()-batch_time)
            break
            
        self.epoch_time = time.time() - start_time
        self.mp.finalize_epoch_metrics(batches)
    
    def postprocess(self,type_):
        #initialize to zero
        pred = []
        y = []
        w_raw = []
        w = []
        cuts = []
        for minibatch in self.kwargs[type_ +"_iterator"]:
            x = minibatch["data"]
            p = self.fns["score"](x)
            pred.extend(p)
            y.extend(minibatch["label"])
            w_raw.extend(minibatch["weight"])
            w.extend(minibatch["normweight"])
            cuts.extend(minibatch["psr"])
        
        #convert to arrays:
        pred=np.asarray(pred,dtype="float32")
        y=np.asarray(y,dtype="int32")
        w_raw=np.asarray(w_raw,dtype="float32")
        w=np.asarray(w,dtype="float32")
        cuts=np.asarray(cuts,dtype="int32")
        
        #call the metrics-processor
        self.mp.process_metrics(type_, pred, y, w_raw, w, cuts, self.epoch_time)
        self.mp.plot_roc_curve(type_, pred, y, w_raw, cuts, self.kwargs["save_path"])